# Worksheet 7 - Classification (Part II)

### Lecture and Tutorial Learning Goals:

After completing this week's lecture and tutorial work, you will be able to:

* Describe what a test data set is and how it is used in classification.
* Using Python, evaluate classification accuracy using a test data set and appropriate metrics.
* Using Python, execute cross-validation in Python to choose the number of neighbours.
* Identify when it is necessary to scale variables before classification and do this using Python
* In a dataset with > 2 attributes, perform k-nearest neighbour classification in Python using the `scikit-learn` package to predict the class of a test dataset.
* Describe advantages and disadvantages of the k-nearest neighbour classification algorithm.

This worksheet covers parts of [Chapter 6](https://python.datasciencebook.ca/classification2) of the online textbook. You should read this chapter before attempting this assignment. Any place you see `___`, you must fill in the function, variable, or data to complete the code. Substitute the `raise NotImplementedError` with your completed code and answers then proceed to run the cell!

In [ ]:
### Run this cell before continuing.
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

**Question 0.1** Multiple Choice:
<br>{points: 1}

Before applying k-nearest neighbour to a classification task, we need to scale the data. What is the purpose of this step?

A. To help speed up the knn algorithm.

B. To convert all data observations to numeric values.

C. To ensure all data observations will be on a comparable scale and contribute equal shares to the calculation of the distance between points.

D. None of the above.

*Assign your answer to an object called `answer0_1`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`)*.

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer0_1)).encode("utf-8")+b"c4e6c868b56f6134").hexdigest() == "f8b9d4acd2ca35e7febd7caf0958dfb451eb4df3", "type of answer0_1 is not str. answer0_1 should be an str"
assert sha1(str(len(answer0_1)).encode("utf-8")+b"c4e6c868b56f6134").hexdigest() == "1b148f9e3d462924c1c4f44694d9c2a6410d353b", "length of answer0_1 is not correct"
assert sha1(str(answer0_1.lower()).encode("utf-8")+b"c4e6c868b56f6134").hexdigest() == "e988777ed908aaddce7b8fb859472b8aae00c101", "value of answer0_1 is not correct"
assert sha1(str(answer0_1).encode("utf-8")+b"c4e6c868b56f6134").hexdigest() == "b3a45bbef71162c1bc7ec8010d00b8b33c250493", "correct string value of answer0_1 but incorrect case of letters"

print('Success!')

## 1. Fruit Data Example - (Part II)
**Question 1.0** 
<br>{points: 1}

You will recognize a few of the first questions in this worksheet from last week's tutorial. This will help you repeat some of the fundamentals of classification before tackling the later questions in this worksheet, which integrate concepts you learned from this week's material. First, load the file `fruit_data.csv` (found in the data folder) from the previous tutorial into your notebook.

*Assign your data to an object called `fruit_data`.*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_data is None)).encode("utf-8")+b"3b194255ef08660a").hexdigest() == "fe574649e9832636a8ee4aa4a641b91003a9f8b5", "type of fruit_data is None is not bool. fruit_data is None should be a bool"
assert sha1(str(fruit_data is None).encode("utf-8")+b"3b194255ef08660a").hexdigest() == "3319b9bb59fde6e482bb5469357961031dba935c", "boolean value of fruit_data is None is not correct"

assert sha1(str(type(fruit_data.shape)).encode("utf-8")+b"3a04cb82dcaf023c").hexdigest() == "4709da702b7a768a334bf1e3c7cc72491bfbede6", "type of fruit_data.shape is not tuple. fruit_data.shape should be a tuple"
assert sha1(str(len(fruit_data.shape)).encode("utf-8")+b"3a04cb82dcaf023c").hexdigest() == "bf536a227ca1a5ba97603a75c63962aee934c404", "length of fruit_data.shape is not correct"
assert sha1(str(sorted(map(str, fruit_data.shape))).encode("utf-8")+b"3a04cb82dcaf023c").hexdigest() == "259226cc3727a5af10227bfe703cab2315ee228a", "values of fruit_data.shape are not correct"
assert sha1(str(fruit_data.shape).encode("utf-8")+b"3a04cb82dcaf023c").hexdigest() == "fa64d7a903116d8702b174e927b903d29bde00ba", "order of elements of fruit_data.shape is not correct"

assert sha1(str(type(fruit_data.fruit_name.dtype)).encode("utf-8")+b"46341806a8fb0b73").hexdigest() == "28d91c35bc79e38b79f391ca3a1f32d2dc24ceba", "type of fruit_data.fruit_name.dtype is not correct"
assert sha1(str(fruit_data.fruit_name.dtype).encode("utf-8")+b"46341806a8fb0b73").hexdigest() == "eb1b2e9cce2a4018f890f3391abbdaea0af0db4c", "value of fruit_data.fruit_name.dtype is not correct"

print('Success!')

Let's take a look at the first few observations in the fruit dataset. Run the cell below.

In [ ]:
# Run this cell.
fruit_data.head()

Run the cell below, and find the nearest neighbour based on mass and width to the first observation just by looking at the scatterplot (the first observation has been circled for you).

In [ ]:
# Create the scatterplot
fruit_chart = alt.Chart(fruit_data).mark_point(size=15).encode(
    x=alt.X("mass").title("Mass (grams)"),
    y=alt.Y("width")
        .title("Width (cm)")
        .scale(zero=False),
    color=alt.Color("fruit_name").title("Fruit")
)
# Create the circle around point1
point1 = alt.Chart().mark_point(size=300, color='black').encode(
    x=alt.datum(fruit_data.loc[0, "mass"]),
    y=alt.datum(fruit_data.loc[0, "width"]),
    text=alt.datum('1')
)

# Layer the scatterplot, point1 circle, and a text annotation
fruit_chart + point1 + point1.mark_text(size=14, dx=12, align='left')

**Question 1.1** Multiple Choice:
<br>{points: 1}

Based on the graph generated, what is the `fruit_name` of the closest data point to the one circled?

A. apple

B. lemon

C. mandarin

D. orange

*Assign your answer to an object called `answer1_1`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_1)).encode("utf-8")+b"7abd4bf8465e1625").hexdigest() == "58a9855cc897d9590e5d49ca4d1cb81b54713eef", "type of answer1_1 is not str. answer1_1 should be an str"
assert sha1(str(len(answer1_1)).encode("utf-8")+b"7abd4bf8465e1625").hexdigest() == "f8a2ba94b8c2e59ef91e0d6a1de5d8421df9900b", "length of answer1_1 is not correct"
assert sha1(str(answer1_1.lower()).encode("utf-8")+b"7abd4bf8465e1625").hexdigest() == "305cc78a46b45d6379a593841940a627c9aecab7", "value of answer1_1 is not correct"
assert sha1(str(answer1_1).encode("utf-8")+b"7abd4bf8465e1625").hexdigest() == "ba2476c4cffe9778b9d0c72af099801088a7b623", "correct string value of answer1_1 but incorrect case of letters"

print('Success!')

**Question 1.2**
<br>{points: 1}

Using `mass` and `width`, calculate the distance between the first observation and the second observation.

We provide a scaffolding to get you started.

*Assign your answer to an object called `fruit_dist_2`.*

In [ ]:
# ___ = euclidean_distances(fruit_data.loc[[___, 1], ["mass", ___]])

# your code here
raise NotImplementedError
fruit_dist_2

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_dist_2)).encode("utf-8")+b"c800fef3748e6b09").hexdigest() == "b8cc3042e50c86921048bab5f56ac7cfc78de6c6", "type of fruit_dist_2 is not correct"
assert sha1(str(fruit_dist_2).encode("utf-8")+b"c800fef3748e6b09").hexdigest() == "a75cfc965843a5132139a9a70e3063575861b4db", "value of fruit_dist_2 is not correct"

print('Success!')

**Question 1.3**
<br>{points: 1}

Calculate the distance between the first and the **44th observation** in the `fruit` dataset using the `mass` and `width` variables.

*Assign your answer to an object called `fruit_dist_44`.*

In [ ]:
# your code here
raise NotImplementedError
fruit_dist_44

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_dist_44)).encode("utf-8")+b"12796f9f94b96106").hexdigest() == "1de0febc561ad82f2683767f9d9ad762296cad22", "type of fruit_dist_44 is not correct"
assert sha1(str(fruit_dist_44).encode("utf-8")+b"12796f9f94b96106").hexdigest() == "dc82e468e9333a47b63a3d37568b8fb37da20522", "value of fruit_dist_44 is not correct"

print('Success!')

What do you notice about your answers from **Question 1.2** & **1.3** that you just calculated? Is it what you would expect given the scatter plot below? Why or why not? Discuss this.

*Hint: Look at where the observations are on the scatterplot in the cell below this question. What might happen if we changed grams into kilograms to measure the mass?*

In [ ]:
# Run this cell to see the scatter plot
points = alt.Chart(
    fruit_data.loc[[0, 1, 43], ['mass', 'width']].assign(point_number=[1, 2, 44])
).mark_point(size=300, color='black').encode(
    x='mass',
    y='width',
    text='point_number'
)

fruit_chart + points + points.mark_text(size=14, dx=12, align='left')

**Question 1.4**
<br>{points: 1}

From the distance calculation, we see that observation 1 and 44 have a smaller distance than observation 1 and 2. However, if we look at the scatterplot the distance of the first observation to the second observation appears closer than to the 44th observation.

Which of the following statements is correct?

A. A difference of 12 g in mass between observation 1 and 2 is large compared to a difference of 1.2 cm in width between observation 1 and 44. Consequently, mass will drive the classification results, and width will have less of an effect. Hence, our distance calculation reflects that.

B. If we measured mass in kilograms, then we’d get different classification results.

C. We should standardize the data so that all variables will be on a comparable scale.

D. All of the above.

*Assign your answer to an object called `answer1_4`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_4)).encode("utf-8")+b"9745ae5230c878d9").hexdigest() == "f012eb87b0d5c62dc31e2d35433ede564c1d2180", "type of answer1_4 is not str. answer1_4 should be an str"
assert sha1(str(len(answer1_4)).encode("utf-8")+b"9745ae5230c878d9").hexdigest() == "27691af6774a470ad82493bd32039561a6122f48", "length of answer1_4 is not correct"
assert sha1(str(answer1_4.lower()).encode("utf-8")+b"9745ae5230c878d9").hexdigest() == "445dcd90c7da2206cde0537a4c383b0f984c39da", "value of answer1_4 is not correct"
assert sha1(str(answer1_4).encode("utf-8")+b"9745ae5230c878d9").hexdigest() == "aba2b4ff813bedcfb086b8df89b88064448a9638", "correct string value of answer1_4 but incorrect case of letters"

print('Success!')

**Question 1.5**
<br> {points: 1}

Let's create a `preprocessor` to *standardize* (i.e., center and scale) all of the variables in the fruit dataset. Centering will make sure that every variable has an average of 0, and scaling will make sure that every variable has standard deviation of 1. We will use the `StandardScaler` in the `preprocessor`. Then `fit_transform` the preprocessor so that we can examine the output.

Fit and transform your preprocessor with predictors `mass`, `width`, `height`, and `color_score`. Pass through all the remaining columns.

Name the preprocessor `fruit_data_preprocessor`, and name the preprocessed data frame `fruit_data_scaled`.

In [ ]:
# ___ = ___(
#     (StandardScaler(), [___, ___, ___, ___]),
#     remainder=___
# )

# ___ = fruit_data_preprocessor.___(___)


# your code here
raise NotImplementedError
fruit_data_scaled.head()

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_data_scaled is None)).encode("utf-8")+b"e5c73cd12b5895f7").hexdigest() == "a96dc8995cac014b0a6cef7cd1f7bf7fb184a2a7", "type of fruit_data_scaled is None is not bool. fruit_data_scaled is None should be a bool"
assert sha1(str(fruit_data_scaled is None).encode("utf-8")+b"e5c73cd12b5895f7").hexdigest() == "40506ea1c3e73f28c97e417b92f9fe52f2540806", "boolean value of fruit_data_scaled is None is not correct"

assert sha1(str(type(fruit_data_scaled.shape)).encode("utf-8")+b"02af27da26fee728").hexdigest() == "320020431b6e57edcfe096a87ba6ad10c646fbe8", "type of fruit_data_scaled.shape is not tuple. fruit_data_scaled.shape should be a tuple"
assert sha1(str(len(fruit_data_scaled.shape)).encode("utf-8")+b"02af27da26fee728").hexdigest() == "8cca7688bd0a459ca38afd3e95594b8c615592f4", "length of fruit_data_scaled.shape is not correct"
assert sha1(str(sorted(map(str, fruit_data_scaled.shape))).encode("utf-8")+b"02af27da26fee728").hexdigest() == "c0022bc43a05a304f94eb3ad779d228f1e4d1006", "values of fruit_data_scaled.shape are not correct"
assert sha1(str(fruit_data_scaled.shape).encode("utf-8")+b"02af27da26fee728").hexdigest() == "d1456ecfe94cb82d5e7b370f1e455698d1ce15cb", "order of elements of fruit_data_scaled.shape is not correct"

assert sha1(str(type(fruit_data_scaled.fruit_name.dtype)).encode("utf-8")+b"8abbfc6a1111f31a").hexdigest() == "9bfe419c3b7b99c4cb125307c94aa6a49be3b968", "type of fruit_data_scaled.fruit_name.dtype is not correct"
assert sha1(str(fruit_data_scaled.fruit_name.dtype).encode("utf-8")+b"8abbfc6a1111f31a").hexdigest() == "1a0f1d96dbe3ce802268e5b6db9baa338db6c4e5", "value of fruit_data_scaled.fruit_name.dtype is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.mass.dropna()))).encode("utf-8")+b"33a1a8f956af1e12").hexdigest() == "82c455fe3dbf54dd0b081863a085dfe581c14064", "type of np.mean(fruit_data_scaled.mass.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.mass.dropna())).encode("utf-8")+b"33a1a8f956af1e12").hexdigest() == "1be8f92352f126a1cf245ffce9836de79f75e021", "value of np.mean(fruit_data_scaled.mass.dropna()) is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.height.dropna()))).encode("utf-8")+b"c2c3984573c11958").hexdigest() == "2c65b5df912158243661e5b46b8e7ab68a511905", "type of np.mean(fruit_data_scaled.height.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.height.dropna())).encode("utf-8")+b"c2c3984573c11958").hexdigest() == "b9970ac475f7083a0d43552d6a2560094343ee71", "value of np.mean(fruit_data_scaled.height.dropna()) is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.width.dropna()))).encode("utf-8")+b"ddb95c3de102596a").hexdigest() == "712d483f8ebb9cbba4882505945336bf9fbefc72", "type of np.mean(fruit_data_scaled.width.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.width.dropna())).encode("utf-8")+b"ddb95c3de102596a").hexdigest() == "20d5ea675f4de35d654d51560df90b9666145326", "value of np.mean(fruit_data_scaled.width.dropna()) is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.color_score.dropna()))).encode("utf-8")+b"a44fd5a211115d31").hexdigest() == "518e4dc642e9137526838c73d7fc1720df721fba", "type of np.mean(fruit_data_scaled.color_score.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.color_score.dropna())).encode("utf-8")+b"a44fd5a211115d31").hexdigest() == "7106e97c9c828041e3bf9015081284c8fbaa2009", "value of np.mean(fruit_data_scaled.color_score.dropna()) is not correct"

assert sha1(str(type(np.std(fruit_data_scaled.mass.dropna()))).encode("utf-8")+b"393022ca525c2308").hexdigest() == "96e9d22319bbde78cf5e06dbf2e190c574ac0664", "type of np.std(fruit_data_scaled.mass.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.mass.dropna()), 2)).encode("utf-8")+b"393022ca525c2308").hexdigest() == "18cbdabdd499f4a9185c1efd27de96f75de7df91", "value of np.std(fruit_data_scaled.mass.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(np.std(fruit_data_scaled.height.dropna()))).encode("utf-8")+b"db4c1b4fc67bddd7").hexdigest() == "b4b6aeb884bda6bdd2b08dac7a9b1e0bb480d927", "type of np.std(fruit_data_scaled.height.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.height.dropna()), 2)).encode("utf-8")+b"db4c1b4fc67bddd7").hexdigest() == "11f20ed4470fabb5b1f077f130e954cf109daaef", "value of np.std(fruit_data_scaled.height.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(np.std(fruit_data_scaled.width.dropna()))).encode("utf-8")+b"0b3737eae218638b").hexdigest() == "6b03965f042cc5b413d58508227ef8853f464c18", "type of np.std(fruit_data_scaled.width.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.width.dropna()), 2)).encode("utf-8")+b"0b3737eae218638b").hexdigest() == "7d3fd17e366d6417799faa81a027f0f4b4cc3975", "value of np.std(fruit_data_scaled.width.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(np.std(fruit_data_scaled.color_score.dropna()))).encode("utf-8")+b"475a7cabdb502ef6").hexdigest() == "1ed48605e0f548d7dd9ceed0c4c5d3acc4430c61", "type of np.std(fruit_data_scaled.color_score.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.color_score.dropna()), 2)).encode("utf-8")+b"475a7cabdb502ef6").hexdigest() == "487f448daebb663d621175107fbd7e5d39980dc6", "value of np.std(fruit_data_scaled.color_score.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(fruit_data_preprocessor is None)).encode("utf-8")+b"0a2ea9c5cbe3201a").hexdigest() == "9f9d4e5cf13c03fb320b8acf8d8ed160330304db", "type of fruit_data_preprocessor is None is not bool. fruit_data_preprocessor is None should be a bool"
assert sha1(str(fruit_data_preprocessor is None).encode("utf-8")+b"0a2ea9c5cbe3201a").hexdigest() == "f4282f5b6d0c1d3b132f1f810cfaaacde70c04ce", "boolean value of fruit_data_preprocessor is None is not correct"

assert sha1(str(type(fruit_data_preprocessor.transformers_[1][2])).encode("utf-8")+b"f37c0e6b332bb9a5").hexdigest() == "287c70bda3a688dfc188464eb0cf6fa8bbeec59a", "type of fruit_data_preprocessor.transformers_[1][2] is not list. fruit_data_preprocessor.transformers_[1][2] should be a list"
assert sha1(str(len(fruit_data_preprocessor.transformers_[1][2])).encode("utf-8")+b"f37c0e6b332bb9a5").hexdigest() == "8fd5a68b4835f44ea46490c03974d48bb9db33ab", "length of fruit_data_preprocessor.transformers_[1][2] is not correct"
assert sha1(str(sorted(map(str, fruit_data_preprocessor.transformers_[1][2]))).encode("utf-8")+b"f37c0e6b332bb9a5").hexdigest() == "18eaee03e8263dbb5c383ea09692475a666e4a0f", "values of fruit_data_preprocessor.transformers_[1][2] are not correct"
assert sha1(str(fruit_data_preprocessor.transformers_[1][2]).encode("utf-8")+b"f37c0e6b332bb9a5").hexdigest() == "5758a068b946d26576a50bf79ae014e8fcff52d5", "order of elements of fruit_data_preprocessor.transformers_[1][2] is not correct"

print('Success!')

**Question 1.6**
<br> {points: 1}

Let's repeat **Question 1.2 and 1.3** with the scaled variables:

- calculate the distance with the scaled mass and width variables between observations 1 and 2
- calculate the distances with the scaled mass and width variables between observations 1 and 44 

After you do this, think about how these distances compared to the distances you computed in **Question 1.2 and 1.3** for the same points.

*Assign your answers to objects called `distance_2` and `distance_44` respectively.*

In [ ]:
# your code here
raise NotImplementedError
print(distance_2)
print(distance_44)

In [ ]:
from hashlib import sha1
assert sha1(str(type(distance_2 is None)).encode("utf-8")+b"54596d681b6ff5bd").hexdigest() == "e61255536d4e6508504739f38f38e92bdfbe9120", "type of distance_2 is None is not bool. distance_2 is None should be a bool"
assert sha1(str(distance_2 is None).encode("utf-8")+b"54596d681b6ff5bd").hexdigest() == "ef3348717dd19706ede349459f2d898fe89867f0", "boolean value of distance_2 is None is not correct"

assert sha1(str(type(distance_44 is None)).encode("utf-8")+b"22678086779e359f").hexdigest() == "4c2851a8b6931f218fca6472d77fae213f00126f", "type of distance_44 is None is not bool. distance_44 is None should be a bool"
assert sha1(str(distance_44 is None).encode("utf-8")+b"22678086779e359f").hexdigest() == "08d3de83842bb7a6896e5f9f103e422817f4f480", "boolean value of distance_44 is None is not correct"

assert sha1(str(type(distance_2)).encode("utf-8")+b"d4f3bd664cf80ede").hexdigest() == "f180d0292aee8e7b5f552c2370d874bdce9f3aad", "type of type(distance_2) is not correct"

assert sha1(str(type(distance_44)).encode("utf-8")+b"2711b66d167b622d").hexdigest() == "1d26b9b90a691109c2c616eb90d14a9cac39c34a", "type of type(distance_44) is not correct"

assert sha1(str(type(distance_2)).encode("utf-8")+b"d5ae27a36a2c2538").hexdigest() == "6fc9a2c39ea144bfd3482f65b28922fc90ebd9f4", "type of distance_2 is not correct"
assert sha1(str(distance_2).encode("utf-8")+b"d5ae27a36a2c2538").hexdigest() == "afc7c533244e9a68fc7032ac6be823fb53245bb4", "value of distance_2 is not correct"

assert sha1(str(type(distance_44)).encode("utf-8")+b"15ab4dd22d431b44").hexdigest() == "b0944c3a7a68e0cea20d6364d3818356cc667997", "type of distance_44 is not correct"
assert sha1(str(distance_44).encode("utf-8")+b"15ab4dd22d431b44").hexdigest() == "c0173f290f1af04fab1299676e61f52323eedc47", "value of distance_44 is not correct"

print('Success!')

## Randomness and Setting Seeds

This worksheet uses functions from the `scikit-learn` library, which not only allows us to perform K-nearest neighbour classification, but also allows us to evaluate how well our classification worked. In order to ensure that the steps in the worksheet are reproducible, we need to set a *`random_state`* or *random seed*, i.e., a numerical "starting value," which determines the sequence of random numbers Python will generate.

Below in many cells we have included an argument to set the `random_state` or `np.random.seed`. They are necessary to make sure the autotesting code functions properly.

## 2. Splitting the data into a training and test set

In this exercise, we will be partitioning `fruit_data` into a training (75%) and testing (25%) set using the `scikit-learn` package. After creating the test set, we will put the test set away in a lock box and not touch it again until we have found the best k-nn classifier we can make using the training set. We will use the variable `fruit_name` as our class label. 


**Question 2.0**
<br> {points: 1}

To create the training and test set, we would use the `train_test_split` function from `scikit-learn` pacakge. Save the trained dataset and test dataset as `fruit_train` and `fruit_test`, respectively. 

In [ ]:
# Randomly take 75% of the data in the training set.
# This will be proportional to the different number of fruit names in the dataset.

# ___, ___ = train_test_split(___, test_size=___, random_state=123) # set the random state to be 123

# your code here
raise NotImplementedError
fruit_train

In [ ]:
fruit_test

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_train is None)).encode("utf-8")+b"24342867dfcccaf2").hexdigest() == "811adc7c9ebdc4344c54e6ec9ec3f832094594af", "type of fruit_train is None is not bool. fruit_train is None should be a bool"
assert sha1(str(fruit_train is None).encode("utf-8")+b"24342867dfcccaf2").hexdigest() == "68c963a77b2c5c8da57f644f744cc1d56ed2861e", "boolean value of fruit_train is None is not correct"

assert sha1(str(type(fruit_test is None)).encode("utf-8")+b"e0cddf4c66a72569").hexdigest() == "7805dab51d5c0e7bbb5fba81014f88494c997500", "type of fruit_test is None is not bool. fruit_test is None should be a bool"
assert sha1(str(fruit_test is None).encode("utf-8")+b"e0cddf4c66a72569").hexdigest() == "2312d407ddb9218bba7db39f37d8499a5cd6f13b", "boolean value of fruit_test is None is not correct"

assert sha1(str(type(fruit_train.shape)).encode("utf-8")+b"06402b6d100e7cbc").hexdigest() == "238abb079d66ebf1fcb2399b7d6c51c421655f60", "type of fruit_train.shape is not tuple. fruit_train.shape should be a tuple"
assert sha1(str(len(fruit_train.shape)).encode("utf-8")+b"06402b6d100e7cbc").hexdigest() == "e1e0a8763752ced476e396729d9b0024c3815955", "length of fruit_train.shape is not correct"
assert sha1(str(sorted(map(str, fruit_train.shape))).encode("utf-8")+b"06402b6d100e7cbc").hexdigest() == "6bf53eb079bf913c8f61d86cd95c09a2464baaaf", "values of fruit_train.shape are not correct"
assert sha1(str(fruit_train.shape).encode("utf-8")+b"06402b6d100e7cbc").hexdigest() == "e6c7ca053d7df65ed13d9bf936158d3c2ec62ed7", "order of elements of fruit_train.shape is not correct"

assert sha1(str(type(fruit_test.shape)).encode("utf-8")+b"b2f74b2b72f872c9").hexdigest() == "0a70a6054198839631fabd3d6c1b918cb28704ff", "type of fruit_test.shape is not tuple. fruit_test.shape should be a tuple"
assert sha1(str(len(fruit_test.shape)).encode("utf-8")+b"b2f74b2b72f872c9").hexdigest() == "a743342bb1a06c20d0a777e9eda5d9c0028aa7d3", "length of fruit_test.shape is not correct"
assert sha1(str(sorted(map(str, fruit_test.shape))).encode("utf-8")+b"b2f74b2b72f872c9").hexdigest() == "be8acd28c2b38762710860ed7bc85e915bca0d4e", "values of fruit_test.shape are not correct"
assert sha1(str(fruit_test.shape).encode("utf-8")+b"b2f74b2b72f872c9").hexdigest() == "4777aaad92b6f528358048442826dc1e4bfaecd3", "order of elements of fruit_test.shape is not correct"

assert sha1(str(type(sum(fruit_train.mass))).encode("utf-8")+b"c2c3f915ae2d2c54").hexdigest() == "2516cb8e7c0f530dc941ffcce939feca9b22798e", "type of sum(fruit_train.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(fruit_train.mass)).encode("utf-8")+b"c2c3f915ae2d2c54").hexdigest() == "741bb248fa1065262793ac10529ec0d0cec514c5", "value of sum(fruit_train.mass) is not correct"

assert sha1(str(type(sum(fruit_test.mass))).encode("utf-8")+b"b5ebf28d54866727").hexdigest() == "738ad299f2b7387e05b7405f657af12b10f003b5", "type of sum(fruit_test.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(fruit_test.mass)).encode("utf-8")+b"b5ebf28d54866727").hexdigest() == "6f423d9c24961ca0d42d6661946e263ad973bc9b", "value of sum(fruit_test.mass) is not correct"

print('Success!')

**Question 2.1** 
<br> {points: 1}

K-nearest neighbors is sensitive to the scale of the predictors so we should do some preprocessing to standardize them. Remember that standardizing involves centering/shifting (subtracting the mean of each variable) and scaling (dividing by its standard deviation). Also remember that standardization is *part of your training procedure*, so you can't use your test data to compute the centered / scaled values for each variable. Therefore, you must pass only the training data to your preprocessor to compute the preprocessing steps. This ensures that our test data does not influence any aspect of our model training. Once we have created the standardization preprocessor, we can then later on apply it separately to both the training and test data sets.

For this exercise, let's see if `mass` and `color_score` can predict `fruit_name`. 

To scale and center the data, first, pass the predictors to the `make_column_transformer` function to make the preprocessor.

*Assign your answer to an object called `fruit_preprocessor`.*

In [ ]:
# ___ = make_column_transformer(
#     (___, [___, ___]),
#     verbose_feature_names_out=False
# )

# your code here
raise NotImplementedError
fruit_preprocessor

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_preprocessor is None)).encode("utf-8")+b"d42b3095d77fb519").hexdigest() == "16561b615f488024931605c5d086b606c92e2d08", "type of fruit_preprocessor is None is not bool. fruit_preprocessor is None should be a bool"
assert sha1(str(fruit_preprocessor is None).encode("utf-8")+b"d42b3095d77fb519").hexdigest() == "cdc1965d0bc271fed44d95ec60b1218be509b29e", "boolean value of fruit_preprocessor is None is not correct"

assert sha1(str(type(type(fruit_preprocessor))).encode("utf-8")+b"8194df6623dc9099").hexdigest() == "3aef8cf41a81932797549a6491ca4bfee5ee1d03", "type of type(fruit_preprocessor) is not correct"
assert sha1(str(type(fruit_preprocessor)).encode("utf-8")+b"8194df6623dc9099").hexdigest() == "a601aa849359aace6a9847de17305b3b51cd839b", "value of type(fruit_preprocessor) is not correct"

assert sha1(str(type(fruit_preprocessor.transformers[0][0])).encode("utf-8")+b"e5902732ea198293").hexdigest() == "710faa6cfa91e1624f140333e5c66f34998699dd", "type of fruit_preprocessor.transformers[0][0] is not str. fruit_preprocessor.transformers[0][0] should be an str"
assert sha1(str(len(fruit_preprocessor.transformers[0][0])).encode("utf-8")+b"e5902732ea198293").hexdigest() == "8b1047d9df29f11d68250a99c4f95e55ca774b30", "length of fruit_preprocessor.transformers[0][0] is not correct"
assert sha1(str(fruit_preprocessor.transformers[0][0].lower()).encode("utf-8")+b"e5902732ea198293").hexdigest() == "fcde4c10e3d8b29733786a13b2ab6e736647f301", "value of fruit_preprocessor.transformers[0][0] is not correct"
assert sha1(str(fruit_preprocessor.transformers[0][0]).encode("utf-8")+b"e5902732ea198293").hexdigest() == "fcde4c10e3d8b29733786a13b2ab6e736647f301", "correct string value of fruit_preprocessor.transformers[0][0] but incorrect case of letters"

assert sha1(str(type(fruit_preprocessor.transformers[0][2])).encode("utf-8")+b"0e063a588d9e77b9").hexdigest() == "c85ca451772f750b7986e67c815f1df0e01cbdfa", "type of fruit_preprocessor.transformers[0][2] is not list. fruit_preprocessor.transformers[0][2] should be a list"
assert sha1(str(len(fruit_preprocessor.transformers[0][2])).encode("utf-8")+b"0e063a588d9e77b9").hexdigest() == "ba56622ececa3355952fb36a645ea67cd76e5826", "length of fruit_preprocessor.transformers[0][2] is not correct"
assert sha1(str(sorted(map(str, fruit_preprocessor.transformers[0][2]))).encode("utf-8")+b"0e063a588d9e77b9").hexdigest() == "e48c7394a590a84a5130493de3fac9690fa17dd4", "values of fruit_preprocessor.transformers[0][2] are not correct"
assert sha1(str(fruit_preprocessor.transformers[0][2]).encode("utf-8")+b"0e063a588d9e77b9").hexdigest() == "143fa1c064dcf44565ca7a0811b5777a06beafc0", "order of elements of fruit_preprocessor.transformers[0][2] is not correct"

print('Success!')

**Question 2.2**
<br> {points: 1}

So far, we have split the training and testing datasets as well as preprocessed the data. Now, let's create our K-nearest neighbour classifier with only the training set using the `scikit-learn` package. First, create the classifier by specifying that we want $K = 3$ neighbors. *Assign your answer to an object called `knn_spec`*. 

Next, separate the predictor columns from the target column. Name the predictor variable `X` and the target `y`. 

Train the classifier with the training data set using the `make_pipeline` and `fit` function. The `make_pipeline` function allows you to bundle together your pre-processing, modeling, and post-processing requests. Scaffolding is provided below for you.

*Assign your answer to an object called `fruit_fit`*.

In [ ]:
# ___ = KNeighborsClassifier(n_neighbors=___)

# ___ = ___[["mass", "color_score"]]
# ___ = fruit_train[___]

# ___ = make_pipeline(___, ___).fit(___, ___)

# your code here
raise NotImplementedError
fruit_fit

In [ ]:
from hashlib import sha1
assert sha1(str(type(knn_spec is None)).encode("utf-8")+b"d4af222d27fb6482").hexdigest() == "9e2c45187ef14f32e088f6a293ce9a9f6e01f83b", "type of knn_spec is None is not bool. knn_spec is None should be a bool"
assert sha1(str(knn_spec is None).encode("utf-8")+b"d4af222d27fb6482").hexdigest() == "a7f316b62a9ce45739c6e1f4e83a098d827e7e6b", "boolean value of knn_spec is None is not correct"

assert sha1(str(type(type(knn_spec))).encode("utf-8")+b"bcc6be2c1a38985e").hexdigest() == "be633027265f760226e8fa4c74031805c662e5bd", "type of type(knn_spec) is not correct"
assert sha1(str(type(knn_spec)).encode("utf-8")+b"bcc6be2c1a38985e").hexdigest() == "d57d78fa5d45517763cd806a1d0b615dbff22498", "value of type(knn_spec) is not correct"

assert sha1(str(type(knn_spec.effective_metric_)).encode("utf-8")+b"3a3edab7640df239").hexdigest() == "3deee27a315e24d0f641f249478678a09e5948ea", "type of knn_spec.effective_metric_ is not str. knn_spec.effective_metric_ should be an str"
assert sha1(str(len(knn_spec.effective_metric_)).encode("utf-8")+b"3a3edab7640df239").hexdigest() == "5c2efd88e8157353b31b45ed8097ba079067bff0", "length of knn_spec.effective_metric_ is not correct"
assert sha1(str(knn_spec.effective_metric_.lower()).encode("utf-8")+b"3a3edab7640df239").hexdigest() == "da7738c2375dba578f2054d32f6535ad3b653355", "value of knn_spec.effective_metric_ is not correct"
assert sha1(str(knn_spec.effective_metric_).encode("utf-8")+b"3a3edab7640df239").hexdigest() == "da7738c2375dba578f2054d32f6535ad3b653355", "correct string value of knn_spec.effective_metric_ but incorrect case of letters"

assert sha1(str(type(knn_spec.n_neighbors)).encode("utf-8")+b"57ab3b502c934247").hexdigest() == "6ba3c4dff2cc3442e5f01f4ede2a40da21bea7c5", "type of knn_spec.n_neighbors is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(knn_spec.n_neighbors).encode("utf-8")+b"57ab3b502c934247").hexdigest() == "5a51c0a5127ca57d6a09a362808e7d7e4103d7b3", "value of knn_spec.n_neighbors is not correct"

assert sha1(str(type(sum(X.mass))).encode("utf-8")+b"fa70a21de506c6dd").hexdigest() == "c8c57205d5bc2dfe40231c887f8eb9ea6f78e517", "type of sum(X.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(X.mass)).encode("utf-8")+b"fa70a21de506c6dd").hexdigest() == "358c95cd1d36cc0fa32c793abaa52c08613400c8", "value of sum(X.mass) is not correct"

assert sha1(str(type(sum(X.color_score))).encode("utf-8")+b"f3d95731f84dee3f").hexdigest() == "e7e415ce4d71c34844210812b1e78927d7bf2bd9", "type of sum(X.color_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(X.color_score), 2)).encode("utf-8")+b"f3d95731f84dee3f").hexdigest() == "ef2bacd38d099f26c0b4ac751ac6f4bb517e975d", "value of sum(X.color_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(y.name)).encode("utf-8")+b"2227bc7e8c51aac5").hexdigest() == "aa88a58bd906a0c1ec6a116665f584703308226d", "type of y.name is not str. y.name should be an str"
assert sha1(str(len(y.name)).encode("utf-8")+b"2227bc7e8c51aac5").hexdigest() == "c153768c2c7f58cf5eee1e84ca6f6e7a84a6aee7", "length of y.name is not correct"
assert sha1(str(y.name.lower()).encode("utf-8")+b"2227bc7e8c51aac5").hexdigest() == "aa3ce7350d24df01a99e7bcfd293c48905cc52cc", "value of y.name is not correct"
assert sha1(str(y.name).encode("utf-8")+b"2227bc7e8c51aac5").hexdigest() == "aa3ce7350d24df01a99e7bcfd293c48905cc52cc", "correct string value of y.name but incorrect case of letters"

assert sha1(str(type(fruit_fit is None)).encode("utf-8")+b"091b2d23467e7eae").hexdigest() == "ac9cd3fcdd8e5344c88f0d43be1b1af862a6ffb6", "type of fruit_fit is None is not bool. fruit_fit is None should be a bool"
assert sha1(str(fruit_fit is None).encode("utf-8")+b"091b2d23467e7eae").hexdigest() == "dd97c84db2674762f35183d22f933e3ae9a3eeac", "boolean value of fruit_fit is None is not correct"

assert sha1(str(type(type(fruit_fit))).encode("utf-8")+b"a26a086c53038974").hexdigest() == "7de676c2d6a7a2c52203e974965943b5cfb213ab", "type of type(fruit_fit) is not correct"
assert sha1(str(type(fruit_fit)).encode("utf-8")+b"a26a086c53038974").hexdigest() == "bee9f81a4ce94e3332ed80325334eeb89ed2dcd0", "value of type(fruit_fit) is not correct"

assert sha1(str(type(len(fruit_fit.named_steps))).encode("utf-8")+b"35e3134df8d9a02f").hexdigest() == "b1edd1db6fda494ef1dca89d8f8a45df4a81a13e", "type of len(fruit_fit.named_steps) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(len(fruit_fit.named_steps)).encode("utf-8")+b"35e3134df8d9a02f").hexdigest() == "478d127fb8070dd243b4deed57166af679177bf5", "value of len(fruit_fit.named_steps) is not correct"

assert sha1(str(type(fruit_fit.named_steps.keys())).encode("utf-8")+b"6297bc0813c8ebd9").hexdigest() == "1281b0fab966c63bf3e115ddf7f12768fc0a47df", "type of fruit_fit.named_steps.keys() is not correct"
assert sha1(str(fruit_fit.named_steps.keys()).encode("utf-8")+b"6297bc0813c8ebd9").hexdigest() == "0c349c59ea744ff92b002d7a55a2a0928bf2d3b7", "value of fruit_fit.named_steps.keys() is not correct"

print('Success!')

**Question 2.3**
<br> {points: 1}

Now that we have created our K-nearest neighbor classifier object, let's predict the class labels for our test set.

We want to make sure to `assign` the predicted class labels to a new column in the dataframe, called `predicted`. To create the predicted class labels pass your fitted model pipeline and the **test dataset** to the `predict` function. 

*Assign your answer to an object called `fruit_test_predictions`.*

In [ ]:
# ___ = fruit_test.___(
#     predicted=___.predict(___[[___, ___]])
# )

# your code here
raise NotImplementedError
fruit_test_predictions

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_test_predictions is None)).encode("utf-8")+b"c04af4fe2a1d73c0").hexdigest() == "f39837135b713094382d761ada719ef33062bdde", "type of fruit_test_predictions is None is not bool. fruit_test_predictions is None should be a bool"
assert sha1(str(fruit_test_predictions is None).encode("utf-8")+b"c04af4fe2a1d73c0").hexdigest() == "870893463fc88c60361e68d1e61a30fac43a96da", "boolean value of fruit_test_predictions is None is not correct"

assert sha1(str(type(fruit_test_predictions)).encode("utf-8")+b"29d54a1aa4d9d672").hexdigest() == "25e5a662dda9c73afca3fc665744d142f1dcf67e", "type of type(fruit_test_predictions) is not correct"

assert sha1(str(type(fruit_test_predictions.shape)).encode("utf-8")+b"34125154dda64c5a").hexdigest() == "d3d1d1f66f91aa89604133ac2494c097475ec657", "type of fruit_test_predictions.shape is not tuple. fruit_test_predictions.shape should be a tuple"
assert sha1(str(len(fruit_test_predictions.shape)).encode("utf-8")+b"34125154dda64c5a").hexdigest() == "21011e44bb93a9f5f621d3d2db44d0c3fdf2504a", "length of fruit_test_predictions.shape is not correct"
assert sha1(str(sorted(map(str, fruit_test_predictions.shape))).encode("utf-8")+b"34125154dda64c5a").hexdigest() == "2c3d49951ac91342e99990144c2e6134a33d4789", "values of fruit_test_predictions.shape are not correct"
assert sha1(str(fruit_test_predictions.shape).encode("utf-8")+b"34125154dda64c5a").hexdigest() == "fe202ce0e40a488c00473b6007b429a30e7a59f0", "order of elements of fruit_test_predictions.shape is not correct"

assert sha1(str(type("predicted" in fruit_test_predictions.columns)).encode("utf-8")+b"1a4a465f1615a04d").hexdigest() == "5a8b4ed8d2d811ffb0dcd171d1e6d45bd6285260", "type of \"predicted\" in fruit_test_predictions.columns is not bool. \"predicted\" in fruit_test_predictions.columns should be a bool"
assert sha1(str("predicted" in fruit_test_predictions.columns).encode("utf-8")+b"1a4a465f1615a04d").hexdigest() == "48f41a45435b7ffd475cf41af9ff9852dd33b9e4", "boolean value of \"predicted\" in fruit_test_predictions.columns is not correct"

print('Success!')

**Question 2.4**
<br> {points: 1}

Great! We have now computed some predictions for our test datasets! From glancing at the dataframe above, it looks like most of them are correct, but wouldn't it be interesting if we could find out the exact accuracy of our classifier? 

Thankfully, the `score` function from the `scikit-learn` package can help us. To get the statistics about the quality of our model, you need to call the `score` function on the `fruit_fit` model. Name the predictors `X_test` and the target `y_test`. We should pass the `X_test` and `y_test` into the `score` function.

*Assign your answer to an object called `fruit_prediction_accuracy`.*

In [ ]:
# ___ = ___[[___, ___]]
# ___ = ___["fruit_name"]

# ___ = fruit_fit.score(___, ___)

# your code here
raise NotImplementedError
fruit_prediction_accuracy

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_prediction_accuracy is None)).encode("utf-8")+b"c53e68bc823bdbab").hexdigest() == "91e0b5db156ed919fd79d0164bb2cbba172dbc73", "type of fruit_prediction_accuracy is None is not bool. fruit_prediction_accuracy is None should be a bool"
assert sha1(str(fruit_prediction_accuracy is None).encode("utf-8")+b"c53e68bc823bdbab").hexdigest() == "c2de87714a547767d2178fabd0e21031e697d864", "boolean value of fruit_prediction_accuracy is None is not correct"

assert sha1(str(type(fruit_prediction_accuracy)).encode("utf-8")+b"eda1e354bf07907e").hexdigest() == "463ac5294a8870a156642ad0e0d4bf37366f08aa", "type of fruit_prediction_accuracy is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(fruit_prediction_accuracy, 2)).encode("utf-8")+b"eda1e354bf07907e").hexdigest() == "1da01776e70824150ef798085ffb9c0eab5f5cac", "value of fruit_prediction_accuracy is not correct (rounded to 2 decimal places)"

print('Success!')

**Question 2.5**
<br> {points: 1}

Now, let's look at the *confusion matrix* for the classifier. This will show us a table comparing the predicted labels with the true labels. 

A confusion matrix is essentially a classification matrix. The columns of the confusion matrix represent the actual class and the rows represent the predicted class (or vice versa). Shown below is an example of a confusion matrix.

|                    | Predicted Positive | Predicted Negative |
|--------------------|:------------------:|:------------------:|
| **Truly Positive** | True Positive      |     False Negative |
| **Truly Negative** | False Positive     |      True Negative |


- A **true positive** is an outcome where the model correctly predicts the positive class.
- A **true negative** is an outcome where the model correctly predicts the negative class.
- A **false positive** is an outcome where the model incorrectly predicts the positive class.
- A **false negative** is an outcome where the model incorrectly predicts the negative class.

<br>

We can create a confusion matrix by using the `crosstab` function from `pandas`. In the dataframe created by `crosstab`, the true labels will be to the left, and the predicted labels will be on top (as in the matrix above). In contrast to the confusion matrix above where there are only two possible outcomes (positive/negative), we have four possible outcomes (the four fruit names). Therefore, our dataframe will be bigger than the matrix above and contain 16 possible outcomes instead of 4.

*Assign your answer to an object called `fruit_mat`*.

In [ ]:
# ___ = pd.___(
#     fruit_test_predictions[___],  # True labels
#     fruit_test_predictions[___],  # Predicted labels
# )

# your code here
raise NotImplementedError
fruit_mat

With many observations, it can be difficult to interpret the confusion matrix when it is presented as a table like above. In these cases, we could instead use the `ConfusionMatrixDisplay` function of the `scikit-learn` package to visualize the confusion matrix as a heatmap. Please run the cell below to see the fruit confusion matrix as a heatmap.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(
    fruit_fit,  # We are directly passing the pipeline and let sklearn do the predictions for us
    X_test,
    y_test
)

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_mat is None)).encode("utf-8")+b"cecea3ca14bb3fbb").hexdigest() == "3f007531d672a7d6887e6ad60c144e17fceedef0", "type of fruit_mat is None is not bool. fruit_mat is None should be a bool"
assert sha1(str(fruit_mat is None).encode("utf-8")+b"cecea3ca14bb3fbb").hexdigest() == "8a6c1a47899a5a8b0ee450b75f8c5f8729407a1e", "boolean value of fruit_mat is None is not correct"

assert sha1(str(type(fruit_mat)).encode("utf-8")+b"39b5a8b79d7ecef4").hexdigest() == "80e27993eb0d86f1acc3b8e71ce34a5948c39d40", "type of type(fruit_mat) is not correct"

assert sha1(str(type(fruit_mat.to_numpy().sum())).encode("utf-8")+b"beb225a1682049b9").hexdigest() == "587f5dc16921a74b2e84273f20e3c68b115e2710", "type of fruit_mat.to_numpy().sum() is not correct"
assert sha1(str(fruit_mat.to_numpy().sum()).encode("utf-8")+b"beb225a1682049b9").hexdigest() == "643d5742737b374bdc67ecebff36057792e6f1d4", "value of fruit_mat.to_numpy().sum() is not correct"

print('Success!')

**Question 2.6** Multiple Choice:
<br> {points: 1}

Reading `fruit_mat`, how many observations were labelled correctly?

A. 7

B. 8

C. 9

D. 14

*Assign your answer to an object called `answer2_6`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError
answer2_6

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer2_6)).encode("utf-8")+b"3f67a04816e6211b").hexdigest() == "8976929979dd46648d2b6ae54120f3e097c188ea", "type of answer2_6 is not str. answer2_6 should be an str"
assert sha1(str(len(answer2_6)).encode("utf-8")+b"3f67a04816e6211b").hexdigest() == "c497c6dd56bec69b487d78706f5c7ca1cccf1d0d", "length of answer2_6 is not correct"
assert sha1(str(answer2_6.lower()).encode("utf-8")+b"3f67a04816e6211b").hexdigest() == "15bda741d8879d1196550bcf3c4de3880b85f95a", "value of answer2_6 is not correct"
assert sha1(str(answer2_6).encode("utf-8")+b"3f67a04816e6211b").hexdigest() == "6929f1a15e137621f8fdf48af94e0d9b08df952d", "correct string value of answer2_6 but incorrect case of letters"

print('Success!')

### 3. Cross-validation

**Question 3.1**
<br> {points: 1}

The vast majority of predictive models in statistics and machine learning have parameters that you have to pick. For the past few exercises, we have had to pick the number of neighbours for the class vote, which we have done arbitraily. But, is it possible to make this selection, *i.e., tune the model, in a principled way?* Ideally, we want to pick the number of neighborurs to maximize the performance of our classifier on data *it hasn’t seen yet*.

An important aspect of the tuning process is that we can, if we want to, split our training data again, train and evaluate a classifier for each split, and then choose the parameter based on all of the different results. If we just split our training data once, our best parameter choice will depend strongly on the randomness from how this single split was made. Using multiple different splits, we’ll get a more robust estimate of accuracy, which will lead to a more suitable choice of the number of neighbours $K$ to perform well on unseen data.

The idea of training and evaluating models on multiple training data splits times is called "cross-validation". In cross-validation, we split our overall training data into $C$ evenly-sized chunks, and then iteratively use 1 chunk as the **validation set** and combine the remaining $C−1$ chunks as the **training set.** The validation set is used in a similar was as the test set, **except** that the test set is only used once at the end to report model performance whether we use model performance on the validation set to select the model during cross-validation. 

---

We can perform a cross-validation in Python using the `cross_validate` function from the `scikit-learn` package. To use this function, you have to identify the model, the training set, and specify the `cv` parameter (the number of folds $C$, defaults to 5). We should set `return_train_score` to be `True` to return the training score as well.

Before we use the `cross_validate` function, we need to perform the pipeline analysis again. You can reuse the `X` and `y` variables you constructed from the training data earlier, as well as the `fruit_preprocessor`and `knn_spec` variables. However, you will need to create a new pipeline since the one we made earlier is already fitted on all the data and here we want to fit it on different splits of the data during cross-validation. Since the cross_validate function outputs a dictionary, we use `pd.DataFrame` to convert it to a dataframe for convenience, as in the textbook.

*Assign your answer to an object called `fruit_vfold_score`*.

In [ ]:
np.random.seed(2020)  # DO NOT REMOVE

# ___ = ___(fruit_preprocessor, knn_spec)
# ___ = pd.___(
#     cross_validate(
#         estimator=___,
#         cv=5,
#         X=___,
#         y=___,
#         return_train_score=True,
#     )
# )

# your code here
raise NotImplementedError
fruit_vfold_score

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_vfold_score is None)).encode("utf-8")+b"1fa574058f03aa42").hexdigest() == "2fff71700ecbd179b6f47faab37bccc0abce426e", "type of fruit_vfold_score is None is not bool. fruit_vfold_score is None should be a bool"
assert sha1(str(fruit_vfold_score is None).encode("utf-8")+b"1fa574058f03aa42").hexdigest() == "9a0f8779f34463a4ebbb83be138f607544e173d6", "boolean value of fruit_vfold_score is None is not correct"

assert sha1(str(type(fruit_vfold_score)).encode("utf-8")+b"81dbd5fc5c1b4fc6").hexdigest() == "f971f1df7ddac29cb452ea5d9528ddc2b156ea8a", "type of type(fruit_vfold_score) is not correct"

assert sha1(str(type(fruit_vfold_score.shape)).encode("utf-8")+b"8157443b918f19e8").hexdigest() == "679eb0bfe62416298c70b561e861da1e6d67f055", "type of fruit_vfold_score.shape is not tuple. fruit_vfold_score.shape should be a tuple"
assert sha1(str(len(fruit_vfold_score.shape)).encode("utf-8")+b"8157443b918f19e8").hexdigest() == "7f87ee2e0541e7f026f73b7d4e5e9b3c0d8d91fb", "length of fruit_vfold_score.shape is not correct"
assert sha1(str(sorted(map(str, fruit_vfold_score.shape))).encode("utf-8")+b"8157443b918f19e8").hexdigest() == "8f91d338d9b1b5c32371a6602eb20d9f25c5f4ae", "values of fruit_vfold_score.shape are not correct"
assert sha1(str(fruit_vfold_score.shape).encode("utf-8")+b"8157443b918f19e8").hexdigest() == "10d2e946a13373e8d4e2948eb8c249151a7c49f3", "order of elements of fruit_vfold_score.shape is not correct"

assert sha1(str(type(fruit_pipe is None)).encode("utf-8")+b"903e81e3470604b6").hexdigest() == "bd0c0221aa377d4d5010b371d1fc96a7bad91b40", "type of fruit_pipe is None is not bool. fruit_pipe is None should be a bool"
assert sha1(str(fruit_pipe is None).encode("utf-8")+b"903e81e3470604b6").hexdigest() == "6219a59427178cb50b60140e8d741bbbfcda262c", "boolean value of fruit_pipe is None is not correct"

assert sha1(str(type(type(fruit_pipe))).encode("utf-8")+b"1cc7719de03f933a").hexdigest() == "3328875d795c2779fe74a15077a4d41dfbd1a5fd", "type of type(fruit_pipe) is not correct"
assert sha1(str(type(fruit_pipe)).encode("utf-8")+b"1cc7719de03f933a").hexdigest() == "dd3c2e572e1222d5254226047f3454a9b13f8a75", "value of type(fruit_pipe) is not correct"

assert sha1(str(type(len(fruit_pipe.named_steps))).encode("utf-8")+b"3723dd23f08a3c22").hexdigest() == "cf6c22a63999e136faf858d71752dfa0eff45218", "type of len(fruit_pipe.named_steps) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(len(fruit_pipe.named_steps)).encode("utf-8")+b"3723dd23f08a3c22").hexdigest() == "a3e343c227dededac48553fb0373ee8926d7d45c", "value of len(fruit_pipe.named_steps) is not correct"

assert sha1(str(type(fruit_pipe.named_steps.keys())).encode("utf-8")+b"2e4322a77d6b96b1").hexdigest() == "7a3bf7d14b2d9bbb153c38ceee9653fa8378a5d2", "type of fruit_pipe.named_steps.keys() is not correct"
assert sha1(str(fruit_pipe.named_steps.keys()).encode("utf-8")+b"2e4322a77d6b96b1").hexdigest() == "032796cacda80d604f8d06ab878037cfd32d255f", "value of fruit_pipe.named_steps.keys() is not correct"

print('Success!')

**Question 3.2**
<br> {points: 1}

Now that we have ran a cross-validation on each train/validation split, one has to ask, how accurate was the classifier's validation across the folds? We can aggregate the *mean* and *standard error* of these scores from each folds. The standard error is essentially a measure of how uncertain we are in the mean value. Use the `agg` dataframe method to compute both the mean and the standard error; make sure the first row of the dataframe contains the mean values and the second contains the standard error values.

*Assign your answer to an object called `fruit_metrics`.*

In [ ]:
# ___ = fruit_vfold_score.___([___, ___])


# your code here
raise NotImplementedError
fruit_metrics

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_metrics.shape)).encode("utf-8")+b"601c6044f2c16e14").hexdigest() == "6e6a9854f2fbe86c1c3be05a70a518ef053c9c7c", "type of fruit_metrics.shape is not tuple. fruit_metrics.shape should be a tuple"
assert sha1(str(len(fruit_metrics.shape)).encode("utf-8")+b"601c6044f2c16e14").hexdigest() == "c2d7a0fd68b24bc8a2ba26e05496f349adfc2e32", "length of fruit_metrics.shape is not correct"
assert sha1(str(sorted(map(str, fruit_metrics.shape))).encode("utf-8")+b"601c6044f2c16e14").hexdigest() == "bd5553416e80763fd127f30076936b76e55c1e15", "values of fruit_metrics.shape are not correct"
assert sha1(str(fruit_metrics.shape).encode("utf-8")+b"601c6044f2c16e14").hexdigest() == "a51123909deb6c4c20c138646e914f01e8181199", "order of elements of fruit_metrics.shape is not correct"

assert sha1(str(type(fruit_metrics)).encode("utf-8")+b"e272ca0ceb19c94c").hexdigest() == "9936ad0b19e31239da72cc8f24d1373af157d659", "type of fruit_metrics is not correct"
assert sha1(str(fruit_metrics).encode("utf-8")+b"e272ca0ceb19c94c").hexdigest() == "bc5bbdc63dd4be0c458633d31582861c8ea78862", "value of fruit_metrics is not correct"

assert sha1(str(type(fruit_metrics.test_score)).encode("utf-8")+b"f52005d17836e005").hexdigest() == "b887db0123aa2d480927f3e522f2ed5d6e833262", "type of fruit_metrics.test_score is not correct"
assert sha1(str(fruit_metrics.test_score).encode("utf-8")+b"f52005d17836e005").hexdigest() == "9eb5b33304c3b15262ea3fe18a9e2831010648dd", "value of fruit_metrics.test_score is not correct"

print('Success!')

## 4. Parameter value selection

Using a 5-fold cross-validation, we have established a prediction accuracy for our classifier. If we were to improve our classifier, we would like to try different number of neighbours, $K$. Then we could use cross-validation to calculate an accuracy for each value of $K$ in a reasonable range, and pick the value of $K$ that gives us the best accuracy on the validation data. 

The great thing about the `scikit-learn` package is that it provides functions to conveniently tune parameters such as $K$ by training and evaluating models (via crossvalidation) for a range of specified values of $K$. The function we will use here is called "exhaustive grid search" ([sklearn.model_selection.GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).

**Question 4.0**
<br> {points: 1}

Create a new K-nearest neighbor model specification but instead of specifying a particular value for the `n_neighbors` argument, try exploring a range of values with `GridSearchCV`. Before we use `GridSearchCV`, we should define the grid of values that we want to explore, and redefine the pipeline without specifying a particular value of $K$. To save us some time, instruct the grid search to use 4-fold cross-validation, rather than the default 5-fold.

*Assign your answer to an object called `knn_tune_grid`.* 

In [ ]:
### Run this cell
param_grid = {
    "kneighborsclassifier__n_neighbors": range(2, 15, 1),
}
fruit_tune_pipe = make_pipeline(fruit_preprocessor, KNeighborsClassifier())

In [ ]:
# ___ = GridSearchCV(
#     ___, ___, ___=__,
# )


# your code here
raise NotImplementedError
knn_tune_grid

In [ ]:
from hashlib import sha1
assert sha1(str(type(knn_tune_grid is None)).encode("utf-8")+b"c143efcc570ff3f6").hexdigest() == "442238e30c09dbbd994d78e9f69de9fc8b6345c6", "type of knn_tune_grid is None is not bool. knn_tune_grid is None should be a bool"
assert sha1(str(knn_tune_grid is None).encode("utf-8")+b"c143efcc570ff3f6").hexdigest() == "711ca705686924589044eafffb8aa7aed8489284", "boolean value of knn_tune_grid is None is not correct"

assert sha1(str(type(type(knn_tune_grid))).encode("utf-8")+b"464473615383e079").hexdigest() == "ae19fd487c9861362064450945d1746385d20366", "type of type(knn_tune_grid) is not correct"
assert sha1(str(type(knn_tune_grid)).encode("utf-8")+b"464473615383e079").hexdigest() == "3e1e44046913c376767b5852ae1eafa3ab988ac0", "value of type(knn_tune_grid) is not correct"

assert sha1(str(type(knn_tune_grid.param_grid.keys())).encode("utf-8")+b"ad4d66ab2952c055").hexdigest() == "8aa531297e78fe9b74349f827c1badd2223f7274", "type of knn_tune_grid.param_grid.keys() is not correct"
assert sha1(str(knn_tune_grid.param_grid.keys()).encode("utf-8")+b"ad4d66ab2952c055").hexdigest() == "4478d5a7eb323a1590721d9c4f6774cf56df95e8", "value of knn_tune_grid.param_grid.keys() is not correct"

assert sha1(str(type(knn_tune_grid.estimator.named_steps.keys())).encode("utf-8")+b"8470947e5d41c3b4").hexdigest() == "0c1b5ed98f9280da4a1e50ca90b1dabf88fa848d", "type of knn_tune_grid.estimator.named_steps.keys() is not correct"
assert sha1(str(knn_tune_grid.estimator.named_steps.keys()).encode("utf-8")+b"8470947e5d41c3b4").hexdigest() == "4942f6d80b061f48e7b7fa5cfb8896fd60494acc", "value of knn_tune_grid.estimator.named_steps.keys() is not correct"

assert sha1(str(type(knn_tune_grid.cv)).encode("utf-8")+b"a076c320aac1726a").hexdigest() == "794f7bb3849c513f66556503fd0591b73c189f73", "type of knn_tune_grid.cv is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(knn_tune_grid.cv).encode("utf-8")+b"a076c320aac1726a").hexdigest() == "45fd7c7861ac7e3c5350198d472f32f2acdc1c9e", "value of knn_tune_grid.cv is not correct"

print('Success!')

**Question 4.1**
<br>{points: 1}

Now, let's fit the grid search object to the data, using the `X` and `y` variables we created earlier.

*Assign your tuned model to a variable called `knn_model_grid`.*

Next, from `knn_model_grid`, find out the `cv_results_` and save it in a dataframe. 

*Assign your answer to a variable called `accuracies_grid`.*

In [ ]:
# ___ = ___.fit(___, ___)

# ___ = pd.DataFrame(___.cv_results_)

# your code here
raise NotImplementedError
accuracies_grid

In [ ]:
from hashlib import sha1
assert sha1(str(type(type(knn_model_grid))).encode("utf-8")+b"ff91e24b91f5b31d").hexdigest() == "a35fbc07f1a7c78d82f05dc6c052f69b0a38ab93", "type of type(knn_model_grid) is not correct"
assert sha1(str(type(knn_model_grid)).encode("utf-8")+b"ff91e24b91f5b31d").hexdigest() == "a89446bb786541b53d83eaf40f097179233717d5", "value of type(knn_model_grid) is not correct"

assert sha1(str(type(accuracies_grid is None)).encode("utf-8")+b"f10a1797a901271d").hexdigest() == "083e0568d94162b3d84d917c8415ab011389b3e3", "type of accuracies_grid is None is not bool. accuracies_grid is None should be a bool"
assert sha1(str(accuracies_grid is None).encode("utf-8")+b"f10a1797a901271d").hexdigest() == "7d9c3d845b4b6b75462eb19c51bb55aa9cbe23f7", "boolean value of accuracies_grid is None is not correct"

assert sha1(str(type(accuracies_grid)).encode("utf-8")+b"968aa4f08a2e2d89").hexdigest() == "bf5c03976b88a751a0daffdab96f99a8752168de", "type of type(accuracies_grid) is not correct"

assert sha1(str(type(accuracies_grid.shape)).encode("utf-8")+b"ed768700e7fbc9b3").hexdigest() == "bbec00988dd1cdea4e1da0b1d8bc952f8004e874", "type of accuracies_grid.shape is not tuple. accuracies_grid.shape should be a tuple"
assert sha1(str(len(accuracies_grid.shape)).encode("utf-8")+b"ed768700e7fbc9b3").hexdigest() == "83952a98b905e9d1941cad224ffb3f0904a7d7c7", "length of accuracies_grid.shape is not correct"
assert sha1(str(sorted(map(str, accuracies_grid.shape))).encode("utf-8")+b"ed768700e7fbc9b3").hexdigest() == "6a5144dc8614e27fc04448359a13132da56d97d3", "values of accuracies_grid.shape are not correct"
assert sha1(str(accuracies_grid.shape).encode("utf-8")+b"ed768700e7fbc9b3").hexdigest() == "a8d48f298e42efb00674e7aafc3ca48e63ae745f", "order of elements of accuracies_grid.shape is not correct"

assert sha1(str(type(sum(accuracies_grid.mean_test_score))).encode("utf-8")+b"805095b28ad587a2").hexdigest() == "4529d8d7431942cc7ad2775c4570d4cb3cae1c14", "type of sum(accuracies_grid.mean_test_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(accuracies_grid.mean_test_score), 2)).encode("utf-8")+b"805095b28ad587a2").hexdigest() == "f227d658fb352981fa94f05619d95983f15398fe", "value of sum(accuracies_grid.mean_test_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(accuracies_grid.std_test_score))).encode("utf-8")+b"cc080c67b31fb5bd").hexdigest() == "32fe6a752210b6b8bb34481e3fdc28b3d5fd428a", "type of sum(accuracies_grid.std_test_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(accuracies_grid.std_test_score), 2)).encode("utf-8")+b"cc080c67b31fb5bd").hexdigest() == "22c3f0c9b939c5595f9ce009bda5c59d8aa72c49", "value of sum(accuracies_grid.std_test_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(accuracies_grid.param_kneighborsclassifier__n_neighbors))).encode("utf-8")+b"676f484973ca9406").hexdigest() == "853624e2bc2f1ac4365772197957799e7f967196", "type of sum(accuracies_grid.param_kneighborsclassifier__n_neighbors) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(accuracies_grid.param_kneighborsclassifier__n_neighbors)).encode("utf-8")+b"676f484973ca9406").hexdigest() == "41c559c37dffd4732f38d77d0c66375aa91243f0", "value of sum(accuracies_grid.param_kneighborsclassifier__n_neighbors) is not correct"

print('Success!')

**Question 4.2**
<br>{points: 1} 


Visually inspecting the grid search results can help us find the best value for the number of neighbors parameter.

Create a line plot using the `accuracies_grid` dataframe with `param_kneighborsclassifier__n_neighbors` on the x-axis and the `mean_test_score` on the y-axis. Use `point=True` to include a point for each value of $K$. Make it an effective visualization.

*Assign your answer to a variable called `accuracy_versus_k_grid`.*

In [ ]:
# ___ = alt.Chart(___).mark_line(___).encode(
#     x=alt.X(___)
#         .title(___)
#         .scale(zero=False),
#     y=alt.Y(___)
#         .title(___)
#         .scale(zero=False)
# )


# your code here
raise NotImplementedError
accuracy_versus_k_grid

In [ ]:
from hashlib import sha1
assert sha1(str(type(accuracy_versus_k_grid is None)).encode("utf-8")+b"7c83b073c0c7345f").hexdigest() == "f56256171217c79d60cd31c5a206d82173799c57", "type of accuracy_versus_k_grid is None is not bool. accuracy_versus_k_grid is None should be a bool"
assert sha1(str(accuracy_versus_k_grid is None).encode("utf-8")+b"7c83b073c0c7345f").hexdigest() == "65037742458a7fb57cf99d25bc242d8b68ca1945", "boolean value of accuracy_versus_k_grid is None is not correct"

assert sha1(str(type(accuracy_versus_k_grid.encoding.x['shorthand'])).encode("utf-8")+b"083dec2ea019efc6").hexdigest() == "871821b85761f911cd549e3748706373608c3452", "type of accuracy_versus_k_grid.encoding.x['shorthand'] is not str. accuracy_versus_k_grid.encoding.x['shorthand'] should be an str"
assert sha1(str(len(accuracy_versus_k_grid.encoding.x['shorthand'])).encode("utf-8")+b"083dec2ea019efc6").hexdigest() == "3b480b32cafe941c125711b7e6b576a0b2976d1a", "length of accuracy_versus_k_grid.encoding.x['shorthand'] is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.x['shorthand'].lower()).encode("utf-8")+b"083dec2ea019efc6").hexdigest() == "51c8c975bf3d35a7bd2a7ffabdbd976c306a721e", "value of accuracy_versus_k_grid.encoding.x['shorthand'] is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.x['shorthand']).encode("utf-8")+b"083dec2ea019efc6").hexdigest() == "51c8c975bf3d35a7bd2a7ffabdbd976c306a721e", "correct string value of accuracy_versus_k_grid.encoding.x['shorthand'] but incorrect case of letters"

assert sha1(str(type(accuracy_versus_k_grid.encoding.y['shorthand'])).encode("utf-8")+b"2cd734f12e2a9b1f").hexdigest() == "bd03abbea01b185f9689c06f9c6ac798f4c4c62f", "type of accuracy_versus_k_grid.encoding.y['shorthand'] is not str. accuracy_versus_k_grid.encoding.y['shorthand'] should be an str"
assert sha1(str(len(accuracy_versus_k_grid.encoding.y['shorthand'])).encode("utf-8")+b"2cd734f12e2a9b1f").hexdigest() == "49b430a73986997b46c37927eac6494d1fcbdc30", "length of accuracy_versus_k_grid.encoding.y['shorthand'] is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.y['shorthand'].lower()).encode("utf-8")+b"2cd734f12e2a9b1f").hexdigest() == "f5263a99780aed072281e552b9696887dcb1e60e", "value of accuracy_versus_k_grid.encoding.y['shorthand'] is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.y['shorthand']).encode("utf-8")+b"2cd734f12e2a9b1f").hexdigest() == "f5263a99780aed072281e552b9696887dcb1e60e", "correct string value of accuracy_versus_k_grid.encoding.y['shorthand'] but incorrect case of letters"

assert sha1(str(type(accuracy_versus_k_grid.mark)).encode("utf-8")+b"5d3152a8031fb0e4").hexdigest() == "e74024b3bc3a0e93278b613c21df662023dec287", "type of accuracy_versus_k_grid.mark is not correct"
assert sha1(str(accuracy_versus_k_grid.mark).encode("utf-8")+b"5d3152a8031fb0e4").hexdigest() == "f857a808cb15cefb97039db018e769cbe01c56e1", "value of accuracy_versus_k_grid.mark is not correct"

assert sha1(str(type(accuracy_versus_k_grid.mark['point'])).encode("utf-8")+b"0edd0242d3832c0c").hexdigest() == "7496f83b14fe310e3526ab237020f9a678f64aa4", "type of accuracy_versus_k_grid.mark['point'] is not bool. accuracy_versus_k_grid.mark['point'] should be a bool"
assert sha1(str(accuracy_versus_k_grid.mark['point']).encode("utf-8")+b"0edd0242d3832c0c").hexdigest() == "a0ce913393190e4bd09f4d9c10609bd98383ac7f", "boolean value of accuracy_versus_k_grid.mark['point'] is not correct"

print('Success!')

From the plots above, we can see that $K = 2$ or $3$ provides the highest accuracy. Larger $K$ values result in a reduced accuracy estimate. Remember: the values you see on this plot are **estimates** of the true accuracy of our classifier. Although this is the best information we have access to for what the ideal value of $K$ would be, it is  not a gurantee that the classifier will always be more accurate with this parameter value when it is used in practice! 

Great, now you have completed a full analysis with cross-validation using the `scikit-learn` package! For your information, we can choose any number of folds and typically, the more we use the better our accuracy estimate will be (lower standard error). However, more folds would mean a greater computation time. In practice, $cv$ is chosen to be either 5 or 10. 